<a href="https://colab.research.google.com/github/AlonaShylkina/The-statistical-significance-of-AB-test-results/blob/main/Statistical_significance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

After conducting A/B tests, it is necessary to check the statistical significance of the results.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from google.colab import drive

drive.mount("/content/drive")
%cd /content/drive/MyDrive/Python_mate

data = pd.read_csv("AB_test.csv")

data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Python_mate


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


In [ ]:
# Define the metrics and their calculations
metrics = {
    "add_payment_info/session": ("add_payment_info", "session"),
    "add_shipping_info/session": ("add_shipping_info", "session"),
    "begin_checkout/session": ("begin_checkout", "session"),
    "new account/session": ("new account", "session"),
}

# Group data by test and test_group
grouped_data = data.groupby(["test", "test_group", "event_name"]).agg({"value": "sum"}).reset_index()

# Prepare results list
results = []

# Loop through each test
for test in grouped_data["test"].unique():
    # Filter data for the current test
    test_data = grouped_data[grouped_data["test"] == test]

    # Split data by test groups
    group_1 = test_data[test_data["test_group"] == 1]
    group_2 = test_data[test_data["test_group"] == 2]

    # Loop through metrics
    for metric_name, (numerator_event, denominator_event) in metrics.items():
        # Calculate numerator and denominator sums for both groups
        numerator_1 = group_1[group_1["event_name"] == numerator_event]["value"].sum()
        denominator_1 = group_1[group_1["event_name"] == denominator_event]["value"].sum()

        numerator_2 = group_2[group_2["event_name"] == numerator_event]["value"].sum()
        denominator_2 = group_2[group_2["event_name"] == denominator_event]["value"].sum()

        # Check for valid denominators to avoid division by zero
        if denominator_1 > 0 and denominator_2 > 0:
            # Perform z-test
            z_stat, p_value = sm.stats.proportions_ztest(
                [numerator_1, numerator_2],
                [denominator_1, denominator_2],
                alternative="two-sided",
            )

            # Calculate conversion rates, changes
            rate_1 = numerator_1 / denominator_1
            rate_2 = numerator_2 / denominator_2
            metric_change = np.round((rate_2 / rate_1 - 1) * 100, 2)

            # Append results
            results.append({
                "Test": test,
                "Metric": metric_name,
                "Numerator_1": numerator_1,
                "Denominator_1": denominator_1,
                "Group1_Conversion_Rate": rate_1,
                "Numerator_2": numerator_2,
                "Denominator_2": denominator_2,
                "Group2_Conversion_Rate": rate_2,
                "Metric Changes, %": metric_change,
                "Z-Stat": z_stat,
                "P-Value": p_value,
                "Significant": p_value < 0.05,
            })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_df.to_csv("stat_results.csv", index=False)

print("Statistical significance analysis complete. \
\nResults saved to 'stat_results.csv' on Google Drive: https://drive.google.com/file/d/1-1xo5BOHa3kHMV0UCJQAnS1_KwrgcMSn/view?usp=sharing.")

Statistical significance analysis complete. 
Results saved to 'stat_results.csv' on Google Drive: https://drive.google.com/file/d/1-1xo5BOHa3kHMV0UCJQAnS1_KwrgcMSn/view?usp=sharing.


The table below shows the statistically validated results of the tests:

In [ ]:
results_df.head(16)

,Test,Metric,Numerator_1,Denominator_1,Group1_Conversion_Rate,Numerator_2,Denominator_2,Group2_Conversion_Rate,"Metric Changes, %",Z-Stat,P-Value,Significant
0,1,add_payment_info/session,1988,45362,0.043825,2229,45193,0.049322,12.54,-3.924884,0.000087,True
1,1,add_shipping_info/session,3034,45362,0.066884,3221,45193,0.071272,6.56,-2.603571,0.009226,True
2,1,begin_checkout/session,3784,45362,0.083418,4021,45193,0.088974,6.66,-2.978783,0.002894,True
3,1,new account/session,3823,45362,0.084278,3681,45193,0.081451,-3.35,1.542883,0.122859,False
4,2,add_payment_info/session,2344,50637,0.046290,2409,50244,0.047946,3.58,-1.240994,0.214608,False
5,2,add_shipping_info/session,3480,50637,0.068724,3510,50244,0.069859,1.65,-0.709557,0.477979,False
6,2,begin_checkout/session,4262,50637,0.084168,4313,50244,0.085841,1.99,-0.952898,0.340642,False
7,2,new account/session,4165,50637,0.082252,4184,50244,0.083274,1.24,-0.588793,0.556000,False
8,3,add_payment_info/session,3623,70047,0.051722,3697,70439,0.052485,1.47,-0.643172,0.520112,False
9,3,add_shipping_info/session,5298,70047,0.075635,5188,70439,0.073652,-2.62,1.413727,0.157442,False


# Conclusions:

**Test #1** shows a statistically significant improvement in add_payment_info, add_shipping_info, and begin_checkout metrics. The new_account metric did not show a statistically significant result but showed a slight decline, which warrants further investigation into the reasons.

**Test #2** did not show statistically significant results for any metrics. It makes sense to continue this test.

**Test #3** only showed a statistically significant result for the begin_checkout metric, and it declined. Therefore, it is not recommended to implement the changes.

**Test #4** showed statistically significant declines in the begin_checkout and new_account metrics, along with non-statistically significant declines in add_payment_info and add_shipping_info. It is recommended to keep the original version without the tested changes.

# Additional:

Tableau Dashboard: https://public.tableau.com/views/ABTests_17336680632730/ABtest?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

